In [1]:
pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os
import sys

import selenium
from selenium import webdriver
import boto3

In [3]:
load_dotenv()

aws_access_key_id = os.getenv('AWS_ID')
aws_secret_access_key = os.getenv('AWS_KEY')
github_token = os.getenv('GITHUB_TOKEN')

s3 = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

In [4]:
proxy_host = os.getenv('PROXY_HOST')
proxy_username = os.getenv('PROXY_USERNAME')
proxy_password = os.getenv('PROXY_PASSWORD')

In [5]:
options = webdriver.ChromeOptions()
options.add_argument("--incognito")
#options.add_argument("--headless")
options.add_argument(f'--proxy-server=http://{proxy_host}')
options.add_argument(f'--proxy-auth={proxy_username}:{proxy_password}')
driver = webdriver.Chrome(options=options)

In [6]:
driver.capabilities['chrome']

{'chromedriverVersion': '119.0.6045.105 (38c72552c5e15ba9b3117c0967a0fd105072d7c6-refs/branch-heads/6045@{#1103})',
 'userDataDir': '/var/folders/2z/_rgstgys255b1k3j7qnrtjrw0000gn/T/.org.chromium.Chromium.JYUL8Q'}

In [7]:
sys.path.insert(0,'/Users/cacau/Documents/GitHub/florianopolis_rent_pricing_monitoring/code/src')

In [8]:
from extractors import Formatter, Extractor
from ingestors import Ingestor

In [ ]:
datestr = datetime.today().date()

In [9]:
def ingest(driver=None, s3=None, all=True, max_pages=None):
    from ingestors import Ingestor

    ingestor = Ingestor(webdriver=driver,
                    cidade='florianopolis',
                    s3=s3) # Instanciando API

    # Chamando a função de ingestão
    ingestor.ingest_pages(
        filename_pattern='page',
        delay_seconds=1.4,
        all=all,
        max_pages=max_pages)
ingest(s3=s3,driver=driver, all=True)

In [10]:
def extract(date=datestr,s3=None):
    from extractors import Extractor

    extractor = Extractor(cidade='florianopolis',
                    s3=s3) # Instanciando API

    folder_base_path = 'pipeline/raw/vivareal/florianopolis/'
    folder_path = f'{folder_base_path}{date}'

    extractor.process_folder(bucket_name = 'floriparentpricing',
                        folder_path=folder_path,
                        filename_pattern='processed',
                        output_format='parquet')
dates = [datestr]
for i in dates:
    extract(date=i,s3=s3)


2023-12-06 13:17:55,377 - INFO - 36 new listings added to result set
2023-12-06 13:17:56,077 - INFO - 35 new listings added to result set
2023-12-06 13:17:57,013 - INFO - 36 new listings added to result set
2023-12-06 13:17:57,982 - INFO - 35 new listings added to result set


KeyboardInterrupt: 

In [ ]:
formatter = Formatter(s3=s3) # Instanciando API

dates = [datestr]
for i in dates:
    formatter.run(
         bucket_name='floriparentpricing',
         datestr=i)

In [11]:
from utils import Aggregator
import pandas as pd
aggregator = Aggregator(s3=s3) # Instanciando API

aggregator.run(
        bucket_name='floriparentpricing', export_method='s3')
df = aggregator.run(
        bucket_name='floriparentpricing', export_method='df')

In [12]:
from utils import GithubApi

api = GithubApi(token=github_token,
                owner='strangercacaus',
                repo='florianopolis_rent_pricing_monitoring',
                branch='main')

api.update_file_content(file_path='data/full_history.csv',
                         new_content = df,
                         method='overwrite')

2023-12-06 13:18:38,127 - INFO - File updated successfully.
